# Import

In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from typing import Any, Dict, Optional, Tuple

import pandas as pd
import numpy as np

import rff

import torch
from torch import nn

import lightning as L
from lightning import LightningDataModule
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader, Dataset, random_split
from torchvision.datasets import MNIST
from torchvision.transforms import transforms

from lightning import LightningModule
from torchmetrics import MinMetric, MeanMetric
from torch.nn.functional import mse_loss
from torchmetrics.classification.accuracy import Accuracy

# Model

In [2]:
class SimpleDenseNet(nn.Module):
    """A simple fully-connected neural net for computing predictions."""

    def __init__(
        self,
        input_size: int = 33,
        lin1_size: int = 64,
        lin2_size: int = 64,
        lin3_size: int = 64,
        output_size: int = 2,
    ) -> None:
        """Initialize a `SimpleDenseNet` module.

        :param input_size: The number of input features.
        :param lin1_size: The number of output features of the first linear layer.
        :param lin2_size: The number of output features of the second linear layer.
        :param lin3_size: The number of output features of the third linear layer.
        :param output_size: The number of output features of the final linear layer.
        """
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(input_size, lin1_size),
            nn.BatchNorm1d(lin1_size),
            nn.ReLU(),
            nn.Linear(lin1_size, lin2_size),
            nn.BatchNorm1d(lin2_size),
            nn.ReLU(),
            nn.Linear(lin2_size, lin3_size),
            nn.BatchNorm1d(lin3_size),
            nn.ReLU(),
            nn.Linear(lin3_size, output_size),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Perform a single forward pass through the network.

        :param x: The input tensor.
        :return: A tensor of predictions.
        """
        batch_size, features = x.size()

        # (batch, 1, width, height) -> (batch, 1*width*height)
        x = x.view(batch_size, -1)

        return self.model(x)


# Dataset

## Loading Dataset

In [3]:
# Input

filename = 'dataset_left_chest_20240627_162502_402545.csv'

In [4]:
# Pandas Dataframe

csv_path = os.path.join(os.path.dirname(os.getcwd()),'data',filename)
df = pd.read_csv(csv_path)
print(len(df))
df.head()

674


,x_c,y_c,z_c,theta_neck_pan_cmd_tminus1,theta_lower_tilt_cmd_tminus1,theta_upper_tilt_cmd_tminus1,theta_left_pan_cmd_tminus1,theta_tilt_cmd_tminus1
0,-0.398367,-0.290066,1.006828,0.0,0.0,0.0,-14,15
1,-0.330193,-0.290690,1.006906,0.0,0.0,0.0,-12,15
2,-0.264120,-0.291673,1.006986,0.0,0.0,0.0,-10,15
3,-0.222958,-0.293428,1.007052,0.0,0.0,0.0,-8,15
4,-0.176419,-0.295422,1.007126,0.0,0.0,0.0,-6,15


## Separation of Training and Validation Set

In [5]:
# Unique Motor Commands

unq_theta_left_pan_cmd = df['theta_left_pan_cmd_tminus1'].unique()
unq_theta_tilt_cmd = df['theta_tilt_cmd_tminus1'].unique()

print('unq_theta_left_pan_cmd:', unq_theta_left_pan_cmd)
print('unq_theta_tilt_cmd:', unq_theta_tilt_cmd)

unq_theta_left_pan_cmd: [-14 -12 -10  -8  -6  -4  -2   0   2   4   6   8  10  12  14]
unq_theta_tilt_cmd: [ 15  10   5   0  -5 -10 -15 -20 -25]


In [6]:
# Separation of Training and Validation Set

# Define the list of choices
val_theta_left_pan = [-14,-10,-6,-2, 2, 6, 10, 14]
train_theta_left_pan = sorted(list(set(unq_theta_left_pan_cmd) - set(val_theta_left_pan)))

print('train_theta_left_pan:', train_theta_left_pan)
print('val_theta_left_pan:', val_theta_left_pan)

train_theta_left_pan: [-12, -8, -4, 0, 4, 8, 12]
val_theta_left_pan: [-14, -10, -6, -2, 2, 6, 10, 14]


In [7]:
# Training Set

train_df = df[df['theta_left_pan_cmd_tminus1'].isin(train_theta_left_pan)].reset_index(drop=True)
print(len(train_df))
train_df.head()

314


,x_c,y_c,z_c,theta_neck_pan_cmd_tminus1,theta_lower_tilt_cmd_tminus1,theta_upper_tilt_cmd_tminus1,theta_left_pan_cmd_tminus1,theta_tilt_cmd_tminus1
0,-0.330193,-0.290690,1.006906,0.0,0.0,0.0,-12,15
1,-0.222958,-0.293428,1.007052,0.0,0.0,0.0,-8,15
2,-0.109503,-0.294611,1.007184,0.0,0.0,0.0,-4,15
3,-0.009580,-0.297689,1.007327,0.0,0.0,0.0,0,15
4,0.100767,-0.298601,1.007452,0.0,0.0,0.0,4,15


In [8]:
# Validation Set

val_df = df[df['theta_left_pan_cmd_tminus1'].isin(val_theta_left_pan)].reset_index(drop=True)
print(len(val_df))
val_df.head()

360


,x_c,y_c,z_c,theta_neck_pan_cmd_tminus1,theta_lower_tilt_cmd_tminus1,theta_upper_tilt_cmd_tminus1,theta_left_pan_cmd_tminus1,theta_tilt_cmd_tminus1
0,-0.398367,-0.290066,1.006828,0.0,0.0,0.0,-14,15
1,-0.264120,-0.291673,1.006986,0.0,0.0,0.0,-10,15
2,-0.176419,-0.295422,1.007126,0.0,0.0,0.0,-6,15
3,-0.056606,-0.296463,1.007262,0.0,0.0,0.0,-2,15
4,0.050289,-0.297829,1.007390,0.0,0.0,0.0,2,15


## Preprocessing

### Degrees to Radians

In [9]:
# Train Set Motor Command Conversion (deg -> rad)

train_df.iloc[:,3:8] = train_df.iloc[:,3:8].apply(np.radians)
train_df.head()

,x_c,y_c,z_c,theta_neck_pan_cmd_tminus1,theta_lower_tilt_cmd_tminus1,theta_upper_tilt_cmd_tminus1,theta_left_pan_cmd_tminus1,theta_tilt_cmd_tminus1
0,-0.330193,-0.290690,1.006906,0.0,0.0,0.0,-0.209440,0.261799
1,-0.222958,-0.293428,1.007052,0.0,0.0,0.0,-0.139626,0.261799
2,-0.109503,-0.294611,1.007184,0.0,0.0,0.0,-0.069813,0.261799
3,-0.009580,-0.297689,1.007327,0.0,0.0,0.0,0.000000,0.261799
4,0.100767,-0.298601,1.007452,0.0,0.0,0.0,0.069813,0.261799


In [10]:
# Validation Set Motor Command Conversion (deg -> rad)

val_df.iloc[:,3:8] = val_df.iloc[:,3:8].apply(np.radians)
val_df.head()

,x_c,y_c,z_c,theta_neck_pan_cmd_tminus1,theta_lower_tilt_cmd_tminus1,theta_upper_tilt_cmd_tminus1,theta_left_pan_cmd_tminus1,theta_tilt_cmd_tminus1
0,-0.398367,-0.290066,1.006828,0.0,0.0,0.0,-0.244346,0.261799
1,-0.264120,-0.291673,1.006986,0.0,0.0,0.0,-0.174533,0.261799
2,-0.176419,-0.295422,1.007126,0.0,0.0,0.0,-0.104720,0.261799
3,-0.056606,-0.296463,1.007262,0.0,0.0,0.0,-0.034907,0.261799
4,0.050289,-0.297829,1.007390,0.0,0.0,0.0,0.034907,0.261799


## Datamodule

In [11]:
# Input

batch_size = 32

### Positional Encoding Example

In [12]:
encoding = rff.layers.PositionalEncoding(sigma=10, m=5)

In [13]:
# Convert the DataFrame to PyTorch tensors

in_train = torch.tensor(train_df.iloc[:,:3].values, dtype=torch.float32)
X1_train = encoding(in_train)
print(X1_train.shape)
X1_train

torch.Size([314, 30])


tensor([[-0.4828, -0.9893,  0.4785,  ..., -0.1826,  0.0538,  0.7972],
        [ 0.1691, -0.6048, -0.9297,  ..., -0.1849,  0.0574,  0.7937],
        [ 0.7725,  0.4621, -0.1568,  ..., -0.1870,  0.0607,  0.7905],
        ...,
        [ 0.7613,  0.4373, -0.2001,  ..., -0.1361, -0.0208,  0.8629],
        [ 0.1565, -0.6207, -0.9174,  ..., -0.1385, -0.0171,  0.8599],
        [-0.4830, -0.9892,  0.4790,  ..., -0.1408, -0.0134,  0.8569]])

In [14]:
X1_train[0,:]

tensor([-0.4828, -0.9893,  0.4785, -0.3944,  0.8659, -0.8757,  0.1460,  0.8781,
        -0.9189, -0.5002, -0.2529, -0.9697, -0.1242,  0.5503,  0.5043, -0.9675,
        -0.2444,  0.9923, -0.8350,  0.8635,  0.9991, -0.8241, -0.9832,  0.9986,
        -0.6037,  0.0434, -0.5664, -0.1826,  0.0538,  0.7972])

In [15]:
X2_train = torch.tensor(train_df.iloc[:,3:6].values, dtype=torch.float32)
print(X2_train.shape)
X2_train

torch.Size([314, 3])


tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0

In [16]:
X_train = torch.cat([X1_train, X2_train], dim=1)
print(X_train.shape)
X_train

torch.Size([314, 33])


tensor([[-0.4828, -0.9893,  0.4785,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1691, -0.6048, -0.9297,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.7725,  0.4621, -0.1568,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.7613,  0.4373, -0.2001,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1565, -0.6207, -0.9174,  ...,  0.0000,  0.0000,  0.0000],
        [-0.4830, -0.9892,  0.4790,  ...,  0.0000,  0.0000,  0.0000]])

### Training Set

In [17]:
# Positional Encoding
in_train = torch.tensor(train_df.iloc[:,:3].values, dtype=torch.float32)
X1_train = encoding(in_train)

# Concatenation
X2_train = torch.tensor(train_df.iloc[:,3:6].values, dtype=torch.float32)
X_train = torch.cat([X1_train, X2_train], dim=1)

# Convert the DataFrame to PyTorch tensors
y_train = torch.tensor(train_df.iloc[:,6:8].values, dtype=torch.float32)

# Create a PyTorch dataset
train_dataset = TensorDataset(X_train, y_train)

# Create a PyTorch data loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=31)

### Validation Set

In [18]:
# Positional Encoding
in_val = torch.tensor(val_df.iloc[:,:3].values, dtype=torch.float32)
X1_val = encoding(in_val)

# Concatenation
X2_val = torch.tensor(val_df.iloc[:,3:6].values, dtype=torch.float32)
X_val = torch.cat([X1_val, X2_val], dim=1)

# Convert the DataFrame to PyTorch tensors
y_val = torch.tensor(val_df.iloc[:,6:8].values, dtype=torch.float32)

# Create a PyTorch dataset
val_dataset = TensorDataset(X_val, y_val)

# Create a PyTorch data loader
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=31)

# Lightning Module

In [19]:
class LeftChestLitModule(LightningModule):
    """Example of a `LightningModule` for MNIST classification.

    A `LightningModule` implements 8 key methods:

    ```python
    def __init__(self):
    # Define initialization code here.

    def setup(self, stage):
    # Things to setup before each stage, 'fit', 'validate', 'test', 'predict'.
    # This hook is called on every process when using DDP.

    def training_step(self, batch, batch_idx):
    # The complete training step.

    def validation_step(self, batch, batch_idx):
    # The complete validation step.

    def test_step(self, batch, batch_idx):
    # The complete test step.

    def predict_step(self, batch, batch_idx):
    # The complete predict step.

    def configure_optimizers(self):
    # Define and configure optimizers and LR schedulers.
    ```

    Docs:
        https://lightning.ai/docs/pytorch/latest/common/lightning_module.html
    """

    def __init__(
        self,
        net: torch.nn.Module,
        optimizer: torch.optim.Optimizer,
        scheduler: torch.optim.lr_scheduler,
        compile: bool,
    ) -> None:
        """Initialize a `MNISTLitModule`.

        :param net: The model to train.
        :param optimizer: The optimizer to use for training.
        :param scheduler: The learning rate scheduler to use for training.
        """
        super().__init__()

        # this line allows to access init params with 'self.hparams' attribute
        # also ensures init params will be stored in ckpt
        self.save_hyperparameters(logger=False)

        self.net = net

        # loss function
        self.criterion = mse_loss

        # for averaging loss across batches
        self.train_loss = MeanMetric()
        self.val_loss = MeanMetric()
        self.test_loss = MeanMetric()

        # for tracking best so far validation accuracy
        self.val_loss_best = MinMetric()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Perform a forward pass through the model `self.net`.

        :param x: A tensor of images.
        :return: A tensor of logits.
        """
        return self.net(x)

    def on_train_start(self) -> None:
        """Lightning hook that is called when training begins."""
        # by default lightning executes validation step sanity checks before training starts,
        # so it's worth to make sure validation metrics don't store results from these checks
        self.val_loss.reset()
        self.val_loss_best.reset()

    def model_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor]
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Perform a single model step on a batch of data.

        :param batch: A batch of data (a tuple) containing the input tensor of images and target labels.

        :return: A tuple containing (in order):
            - A tensor of losses.
            - A tensor of predictions.
            - A tensor of target labels.
        """
        x, y = batch
        preds = self.forward(x)
        loss = self.criterion(preds, y)
        return loss, preds, y

    def training_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_idx: int
    ) -> torch.Tensor:
        """Perform a single training step on a batch of data from the training set.

        :param batch: A batch of data (a tuple) containing the input tensor of images and target
            labels.
        :param batch_idx: The index of the current batch.
        :return: A tensor of losses between model predictions and targets.
        """
        loss, preds, targets = self.model_step(batch)

        # update and log metrics
        self.train_loss(loss)
        self.log("train/loss", self.train_loss, on_step=False, on_epoch=True, prog_bar=True)

        # return loss or backpropagation will fail
        return loss

    def on_train_epoch_end(self) -> None:
        "Lightning hook that is called when a training epoch ends."
        pass

    def validation_step(self, batch: Tuple[torch.Tensor, torch.Tensor], batch_idx: int) -> None:
        """Perform a single validation step on a batch of data from the validation set.

        :param batch: A batch of data (a tuple) containing the input tensor of images and target
            labels.
        :param batch_idx: The index of the current batch.
        """
        loss, preds, targets = self.model_step(batch)

        # update and log metrics
        self.val_loss(loss)
        # print("val/loss:", self.val_loss.compute())  # print
        self.log("val/loss", self.val_loss, on_step=False, on_epoch=True, prog_bar=True)

    def on_validation_epoch_end(self) -> None:
        "Lightning hook that is called when a validation epoch ends."
        loss = self.val_loss.compute()  # get current val acc
        self.val_loss_best(loss)  # update best so far val acc
        # log `val_acc_best` as a value through `.compute()` method, instead of as a metric object
        # otherwise metric would be reset by lightning after each epoch
        print("val/loss_best:", self.val_loss_best.compute())  # print
        self.log("val/loss_best", self.val_loss_best.compute(), sync_dist=True, prog_bar=True)

    def test_step(self, batch: Tuple[torch.Tensor, torch.Tensor], batch_idx: int) -> None:
        """Perform a single test step on a batch of data from the test set.

        :param batch: A batch of data (a tuple) containing the input tensor of images and target
            labels.
        :param batch_idx: The index of the current batch.
        """
        loss, preds, targets = self.model_step(batch)

        # update and log metrics
        self.test_loss(loss)
        self.log("test/loss", self.test_loss, on_step=False, on_epoch=True, prog_bar=True)

    def on_test_epoch_end(self) -> None:
        """Lightning hook that is called when a test epoch ends."""
        pass

    def setup(self, stage: str) -> None:
        """Lightning hook that is called at the beginning of fit (train + validate), validate,
        test, or predict.

        This is a good hook when you need to build models dynamically or adjust something about
        them. This hook is called on every process when using DDP.

        :param stage: Either `"fit"`, `"validate"`, `"test"`, or `"predict"`.
        """
        if self.hparams.compile and stage == "fit":
            self.net = torch.compile(self.net)

    def configure_optimizers(self) -> Dict[str, Any]:
        """Choose what optimizers and learning-rate schedulers to use in your optimization.
        Normally you'd need one. But in the case of GANs or similar you might have multiple.

        Examples:
            https://lightning.ai/docs/pytorch/latest/common/lightning_module.html#configure-optimizers

        :return: A dict containing the configured optimizers and learning-rate schedulers to be used for training.
        """
        # optimizer = self.hparams.optimizer(params=self.trainer.model.parameters())
        optimizer = torch.optim.Adam(self.parameters(), lr=5e-3)
        if self.hparams.scheduler is not None:
            scheduler = self.hparams.scheduler(optimizer=optimizer)
            return {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": scheduler,
                    "monitor": "val/loss",
                    "interval": "epoch",
                    "frequency": 1,
                },
            }
        return {"optimizer": optimizer}


# Training

In [20]:
# Initialization

dense_net = LeftChestLitModule(SimpleDenseNet(), None, None, False)
trainer = L.Trainer(min_epochs=1, max_epochs=100)

/home/jaynieles/dev/aec/venv/lib/python3.8/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [21]:
# Training

trainer.fit(model=dense_net, train_dataloaders=train_loader, val_dataloaders=val_loader)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type           | Params | Mode 
---------------------------------------------------------
0 | net           | SimpleDenseNet | 11.0 K | train
1 | train_loss    | MeanMetric     | 0      | train
2 | val_loss      | MeanMetric     | 0      | train
3 | test_loss     | MeanMetric     | 0      | train
4 | val_loss_best | MinMetric      | 0      | train
---------------------------------------------------------
11.0 K    Trainable params
0         Non-trainable params
11.0 K    Total params
0.044     Total estimated model params size (MB)


Sanity Checking: |                                                                                            …

val/loss_best: tensor(0.0252, device='cuda:0')


/home/jaynieles/dev/aec/venv/lib/python3.8/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                                   …

...

Validation: |                                                                                                 …

val/loss_best: tensor(0.0089, device='cuda:0')


Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=100` reached.


val/loss_best: tensor(0.0089, device='cuda:0')
